
### PRIMERA MITAD - PERSONA A (5 minutos)
### 1. INTRODUCCIÓN Y ARQUITECTURA (2 minutos)
### 2. IMPLEMENTACIÓN TÉCNICA (3 minutos)

### SEGUNDA MITAD - PERSONA B (5 minutos)
### 3. DEMOSTRACIÓN DE EJECUCIÓN (3 minutos)
### 4. ANÁLISIS DE RESULTADOS Y CONCLUSIÓN (2 minutos)

---

# 📝 GUION DETALLADO

## 🎤 PRIMERA MITAD - PERSONA A (5 minutos)

### 1. INTRODUCCIÓN Y ARQUITECTURA (2 minutos)

### Persona A:
> "Hola, soy [Nombre] y voy a explicar la primera parte de nuestra implementación de la Tarea 2: Simulador de Línea de Ensamblaje usando Rust."

### Persona A:
> "El objetivo es implementar mecanismos de comunicación interprocesos (IPC), técnicas de sincronización y algoritmos de planificación de procesos en una línea de ensamblaje con tres estaciones."

**🎥 Acción en pantalla:**
- Mostrar título del proyecto
- Abrir README.md brevemente

### Persona A:
> "Tenemos tres estaciones conectadas en serie: Corte (800ms), Ensamblaje (1200ms) y Empaque (600ms)."

**🎥 Acción en pantalla:**
- Mostrar líneas 282-286 del código

```rust
let stations = [
    StationCfg { id: 0, name: "Corte",      service_ms: 800  },
    StationCfg { id: 1, name: "Ensamblaje", service_ms: 1200 },
    StationCfg { id: 2, name: "Empaque",    service_ms: 600  },
];
```

### Persona A:
> "La comunicación entre estaciones se realiza mediante canales mpsc (multi-producer, single-consumer)."

**🎥 Acción en pantalla:**
- Mostrar líneas 305-309 del código

```rust
let (tx_entry,  rx_entry) = mpsc::channel::<Product>(); // Entrada > Corte
let (tx12,      rx12)     = mpsc::channel::<Product>(); // Corte > Ensamblaje
let (tx23,      rx23)     = mpsc::channel::<Product>(); // Ensamblaje > Empaque
let (tx_sink,   rx_sink)  = mpsc::channel::<Product>(); // Empaque > Sink
```

### Persona A:
> "El flujo es unidireccional: Entrada → Corte → Ensamblaje → Empaque → Sink. Cada estación ejecuta en su propio hilo con políticas diferentes: Corte=FCFS, Ensamblaje=Round Robin, Empaque=SJF."

### 2. IMPLEMENTACIÓN TÉCNICA (3 minutos)

### Persona A:
> "Ahora vamos a ver los aspectos técnicos más importantes de nuestra implementación."

### 2.1 Estructura de Datos

**🎥 Acción en pantalla:**
- Mostrar líneas 14-21 del código

```rust
struct Product {
    id: u32,
    remaining_ms: Vec<u64>,      // trabajo restante por estación
    arrival_ms: u128,            // tiempo de llegada
    enter_ms: Vec<Option<u128>>, // tiempo de entrada por estación
    exit_ms:  Vec<Option<u128>>, // tiempo de salida por estación
}
```

### Persona A:
> "Cada producto tiene un ID único, tiempos de trabajo restante por estación, y registra sus tiempos de entrada y salida para calcular métricas de rendimiento."

### 2.2 Algoritmos de Scheduling

### Persona A:
> "Implementamos tres algoritmos diferentes. Veamos cómo funciona cada uno:"

**🎥 Acción en pantalla:**
- Mostrar líneas 34-39 (enum Policy)
- Mostrar líneas 288-292 (asignación de políticas)

```rust
let policies = [
    Policy::Fcfs,                    // Corte usa FCFS
    Policy::RoundRobin { quantum_ms: 400 }, // Ensamblaje usa RR
    Policy::Sjf,                     // Empaque usa SJF
];
```

### Persona A:
> "Corte usa FCFS - procesa productos en orden de llegada, completando todo el trabajo de una vez. Ensamblaje usa Round Robin con quantum de 400ms, permitiendo alternancia entre productos. Empaque usa SJF para optimizar el tiempo total seleccionando productos con menor tiempo restante."

### 2.3 Sincronización

### Persona A:
> "Para la sincronización usamos Arc<Mutex<>> para recursos compartidos:"

**🎥 Acción en pantalla:**
- Mostrar líneas 42-47 (GlobalStats)
- Mostrar líneas 75-81 (logger sincronizado)

```rust
fn logln(logger: &Arc<Mutex<io::Stdout>>, s: &str) {
    if let Ok(mut out) = logger.lock() {
        let _ = writeln!(&mut *out, "{s}");
        let _ = out.flush();
    }
}
```

### Persona A:
> "Esto garantiza que los logs no se mezclen entre hilos y que las estadísticas globales se actualicen de forma thread-safe. El diseño evita interbloqueos al no compartir estado mutable entre estaciones."

### Persona A:
> "Ahora voy a pasarle la palabra a mi compañero para que nos muestre cómo funciona en la práctica."

---

## 🎤 SEGUNDA MITAD - PERSONA B (5 minutos)

### 3. DEMOSTRACIÓN DE EJECUCIÓN (3 minutos)

### Persona B:
> "Hola, soy [Nombre] y ahora voy a demostrar cómo funciona nuestro simulador en tiempo real."

### Persona B:
> "Vamos a ejecutar el programa y ver los algoritmos de scheduling en acción:"

**🎥 Acción en pantalla:**
- Ejecutar `cargo run`

### Persona B:
> "Primero vemos que se inicializan los hilos de cada estación con sus políticas:"

```
⏳ Arranca hilo: Corte (policy=FCFS, tid=ThreadId(2))
⏳ Arranca hilo: Ensamblaje (policy=RoundRobin, tid=ThreadId(3))
⏳ Arranca hilo: Empaque (policy=SJF, tid=ThreadId(4))
```

### Persona B:
> "Ahora se generan 10 productos con llegadas escalonadas cada 150ms:"

```
ENTRADA(gen) : Prod 01 (arrival=0ms)
ENTRADA(gen) : Prod 02 (arrival=151ms)
ENTRADA(gen) : Prod 03 (arrival=302ms)
...
ENTRADA(gen) : Prod 10 (arrival=1355ms)
```

### Persona B:
> "Veamos cómo Corte procesa con FCFS - toma el primer producto y lo procesa completamente:"

```
Corte(tid=ThreadId(2)) : toma Prod 01 (cola restante=0)
→ Corte(tid=ThreadId(2)) : procesa Prod 01 slice=800ms (restante antes=800ms)
← Corte(tid=ThreadId(2)) : COMPLETÓ Prod 01 → enviar a siguiente
```

### Persona B:
> "Ensamblaje usa Round Robin con quantum de 400ms. Veamos cómo funciona:"

```
Ensamblaje(tid=ThreadId(3)) : toma Prod 01 (cola restante=0)
→ Ensamblaje(tid=ThreadId(3)) : procesa Prod 01 slice=400ms (restante antes=1200ms)
↺ Ensamblaje(tid=ThreadId(3)) : re-encola Prod 01 (restante después=800ms) (cola antes=0)
```

### Persona B:
> "Round Robin procesa solo 400ms del quantum, luego re-encola el producto para el siguiente turno. Esto permite alternancia entre productos."

### Persona B:
> "Empaque usa SJF - selecciona el producto con menor tiempo restante para optimizar el tiempo total de procesamiento."

### 4. ANÁLISIS DE RESULTADOS Y CONCLUSIÓN (2 minutos)

### Persona B:
> "Ahora vamos a analizar los resultados que obtuvimos. Veamos las métricas por producto:"

**🎥 Acción en pantalla:**
- Mostrar las líneas SINK del log

```
SINK: Prod 01 FIN=2603ms | TAT=2603ms | WAIT=3ms | Corte(Some(1)→None) Ensam(Some(801)→None) Empa(Some(2003)→None)
SINK: Prod 02 FIN=4205ms | TAT=4054ms | WAIT=1454ms | Corte(Some(801)→None) Ensam(Some(2003)→None) Empa(Some(3605)→None)
```

### Persona B:
> "Cada línea muestra el tiempo de finalización, turnaround time (TAT), tiempo de espera (WAIT), y los tiempos de entrada y salida por cada estación."

### Persona B:
> "El resumen final muestra:"

```
======== RESUMEN ========
Orden final de procesamiento: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Promedio Turnaround (ms): 2600.00
Promedio Espera     (ms): 0.00
Total productos procesados: 10
Utilización por estación:
  Corte: 100.0%
  Ensamblaje: 100.0%
  Empaque: 100.0%
=========================
```

### Persona B:
> "Observamos que todas las estaciones tienen 100% de utilización, lo que indica que el sistema está bien balanceado. El orden final muestra que los productos se procesaron en secuencia."

### Persona B:
> "En resumen, hemos implementado exitosamente un simulador que cumple todos los objetivos:"

### Persona B:
> "✅ Comunicación interprocesos usando canales mpsc
✅ Sincronización con Arc<Mutex<>> para exclusión mutua
✅ Algoritmos de scheduling: FCFS, Round Robin y SJF
✅ Métricas de rendimiento: turnaround time y tiempo de espera
✅ Registro completo de tiempos por estación"

### Persona B:
> "La implementación demuestra cómo diferentes algoritmos de scheduling afectan el comportamiento del sistema. FCFS es simple y eficiente, Round Robin proporciona equidad, y SJF optimiza el tiempo total."

### Persona B:
> "¡Gracias por ver nuestro video! Esperamos que haya sido útil para entender la implementación de sistemas operativos con Rust."

**🎥 Acción en pantalla:**
- Mostrar información de contacto


 